In [ ]:
import pandas
events = pandas.read_csv('data/events.csv', sep=';', skiprows=1)
persons = pandas.read_csv('data/persons.csv', sep=';', skiprows=1)
documents = pandas.read_csv('data/documents.csv', sep=';', skiprows=1)

gedcom = GedcomReader().read_gedcom('data/exmp.ged')
print(len(gedcom.persons))
for family in gedcom.families:
    print(family)

In [2]:
import upload
archive = upload.create_package(r'D:\site', 'data')
print(archive)
# scp data.zip kzhagorina@ssh.pythonanywhere.com:~/genotree/upload/data.zip

D:\site\data.zip


In [ ]:
def split_binary_file(filename, chunk_size=50*1024*1024):
    """split file into chunks of size"""
    
    f = open(filename, 'rb')
    data = f.read()
    f.close()

    count_chunks = int(len(data)/chunk_size + bool(len(data)%chunk_size))
    for i in range(0, count_chunks):
        chunk_filename = filename+".{0:000}".format(i+1)
        f = open(chunk_filename, 'wb+')
        f.write(data[i*chunk_size: (i+1)*chunk_size])
        f.close()

In [ ]:

split_binary_file(r'D:\site\data.zip')

In [1]:
import upload
archive = r'D:\site\data.zip'
upload.load_package(archive, 
                    r'C:\Users\Ksenia\Documents\IPython Notebooks\genotree\src\static\tree',
                    r'C:\Users\Ksenia\Documents\IPython Notebooks\genotree\data\tree')

# cd genotree/src
# python3
# import upload
# archive = '../upload/data.zip'
# upload.load_package(archive, 'static/tree', '../data/tree')

undefined: [PAGE http://ok.ru/profile/121643311] in {}
Date of marriage in gedcom: None date in xml: <Element 'date' at 0x00000000065FB5E8> (BEF 2009)
Comment of marriage in gedcom: None comment in xml: В 2009-2015 годах вместе построили большой дом (2 этажа + подвал) на окраине г. Екатеринбурга (ул. Байдукова, 67)
Place of marriage in gedcom: None place in xml: г.Екатеринбург
Marriage of persons RFgpru4rw1 and iyAHSSNXVq does not valid
Date of marriage in gedcom: 10 SEP 1966 date in xml: <Element 'date' at 0x00000000092FEBD8> (APR 1972)
Comment of marriage in gedcom: None comment in xml: познакомились, когда т.Тамара работала в Н.Серги в воинской части экономистом, а Геннадий работал там же в отделе кадров
Place of marriage in gedcom: г.Среднеуральск place in xml: г. Нижние Серги-3
Marriage of persons hmff4GlxxT and hmCbFvMhLQ does not valid


In [ ]:
from gedcom import Gedcom, GedcomReader, GedcomWriter
gedcom = GedcomReader().read_gedcom('D:/site/tree.ged')

In [ ]:
gedcom = GedcomReader().read_gedcom('data/exmp.ged')